In [1]:
import numpy as np
import itertools
import cvxpy as cp

def random_decoding_orders(U, num_orders):
    """
    Generate a list of candidate decoding orders for each receiver.
    Each decoding order is a list (or tuple) of length U^2 with sub-stream indices (j,k).
    produce random 
    """

def compute_sinr(i, k, p, h, sigma, decode_order):
    """
    Compute the SINR of sub-stream (i,k) at receiver i given:
      - p: 2D array of powers p[j,k]
      - h[i,j]: channel gain from user j to receiver i
      - sigma[i]: noise power at receiver i
      - decode_order: a single permutation (list of sub-streams) for receiver i
        (Assuming we are applying the same order for all i in this toy example,
         but you can store a separate decode_order_i if needed.)
    The sub-stream (i,k) is decoded at some position in decode_order.
    Interference is from any sub-stream that is decoded at the same or later position.
    """
    # Find position (index) of (i,k) in decode_order
    idx = decode_order.index((i,k))
    # sub-streams that appear from idx onward are not yet cancelled
    # so they remain as interference
    interfering_subs = decode_order[idx:]
    
    numerator = p[i, k] * h[i, i]
    denominator = sigma[i]
    for (m,n) in interfering_subs:
        if (m,n) != (i,k):
            denominator += p[m, n] * h[i, m]
    sinr = numerator / denominator
    return sinr

def compute_rates(p, h, sigma, decode_orders, U):
    """
    Given a power matrix p of shape (U, U), decode orders, etc.,
    return a matrix R of shape (U, U) with R[i,k] = rate of sub-stream (i,k).
    """
    R = np.zeros((U, U))
    for i in range(U):
        decode_order_i = decode_orders[i]  # if you have per-receiver orders
        for k in range(U):
            sinr_ik = compute_sinr(i, k, p, h, sigma, decode_order_i)
            R[i, k] = np.log2(1.0 + sinr_ik)
    return R

def total_rate_of_user(i, R):
    return np.sum(R[i,:])

def check_feasibility(p, h, sigma, decode_orders, R_min, U):
    """
    Compute rates. Check if each user's sum-rate >= R_min[i].
    Return (feasible_bool, R_matrix).
    """
    R = compute_rates(p, h, sigma, decode_orders, U)
    for i in range(U):
        if total_rate_of_user(i, R) < R_min[i]:
            return (False, R)
    return (True, R)

def solve_power_allocation_sca(h, sigma, R_min, decode_orders, 
                               max_iter=30, tol=1e-3):
    """
    A simple Successive/Iterative approach to find a feasible power matrix p.
    
    We'll keep the maximum power in some finite bound for demonstration.
    We'll set an initial guess p_init, then refine it.
    This is not a fully fledged SCA, but an illustrative iterative approach:
      1) Fix interference from last iteration
      2) Solve a "convexified" or simpler subproblem
      3) Update p
      4) Repeat until convergence or max_iter
    
    For real systems, you'd want a more robust approach (e.g. genuine SCA).
    """
    U = len(R_min)
    # Let's define an upper bound on power for each sub-stream, e.g. 10.0
    p_max = 10.0
    
    # Start with a random or uniform power
    p_current = np.ones((U, U)) * 0.1
    
    for iteration in range(max_iter):
        # We'll define a simple approach using CVXPY:
        p_var = cp.Variable((U, U), nonneg=True)
        
        # We want to minimize sum of p_var
        objective = cp.Minimize(cp.sum(p_var))
        
        constraints = []
        # For each user i, the sum of rates >= R_min[i].
        # We'll linearize the rate constraints in a simple manner:
        
        # Rate ~ log2(1 + p[i,k]*h[i,i]/(sigma_i + sum_{(m,n) in I_{i,k}} p[m,n]*h[i,m]))
        # This is the tricky part (nonconvex). Let's do a first-order approximation
        # around p_current (for demonstration).
        
        # Step 1: compute the baseline denominators at p_current
        # to get the SINR, then linearize log(1+ x/(D)) ~ log(1+ x0/D0) + ...
        
        # We'll store the partial derivatives to linearize each sub-stream's rate.
        sinr_current = np.zeros((U,U))
        
        # For each receiver i, sub-stream k, find the "not-yet-canceled" set:
        for i_ in range(U):
            decode_order_i = decode_orders[i_]
            for k_ in range(U):
                # Evaluate sinr at p_current
                sinr_current[i_, k_] = compute_sinr(i_, k_, p_current, h, sigma, decode_order_i)
        
        # Now we build constraints for each user i:
        for i_ in range(U):
            # sum_{k} R_{i_,k} >= R_min[i_]
            # We'll define a variable for sum of rates, then do R_{i_,k} >= some linear approximation.
            # For simplicity, let's do a "conservative" linear lower bound on each sub-stream's rate:
            #   R = log2(1 + sinr).
            # The derivative w.r.t p_{m,n} is complicated, but let's do a simpler approach:
            
            # We'll just say: "Rate_i >= log2(1 + sinr_current(i_,k_))" to keep a naive solution,
            # ignoring the derivative. This is obviously an *under*-approx, so might be feasible but suboptimal.
            
            # For better performance, you'd do a proper partial derivative-based linearization.
            
            # We'll sum these "constant" approximations as a single constraint to ensure
            # sum of rates >= R_min. This is a big underestimate, but it’s simple for demonstration.
            
            rate_lower_bound = 0.0
            decode_order_i = decode_orders[i_]
            for k_ in range(U):
                rate_lower_bound += np.log2(1.0 + sinr_current[i_, k_])
            
            constraints += [
                rate_lower_bound >= R_min[i_]
            ]
        
        # Also ensure p_var <= p_max
        constraints += [p_var <= p_max]
        
        # Solve this simplistic problem
        prob = cp.Problem(objective, constraints)
        prob.solve(solver=cp.MOSEK, warm_start=True)  # or another solver
        
        if p_var.value is None:
            # Infeasible in this linear approx
            # We can break or increase p_max or refine approach
            return None
        
        # Update p_current
        p_next = p_var.value
        
        # Check if we've converged
        diff = np.linalg.norm(p_next - p_current)
        p_current = p_next
        if diff < tol:
            break
    
    return p_current

def solve_minPIC(U, h, sigma, R_min, num_order_candidates=10):
    """
    Main wrapper:
      1) Generate candidate decoding orders for each receiver i (or shared).
      2) For each candidate, run a SCA/iterative solver to get a feasible p.
      3) Check feasibility and record sum-power if feasible.
      4) Return the best (lowest sum-power) solution found.
    """
    # For simplicity: assume each receiver uses the *same* decoding order in this example
    # You can generalize by generating a random decoding order for *each* receiver i.
    candidate_orders = []
    for _ in range(num_order_candidates):
        # We'll create a separate random order for each receiver i
        # So candidate_orders is a list of length num_order_candidates,
        # where each element is a list of orders [perm_0, perm_1, ..., perm_(U-1)].
        decode_orders_for_all = []
        all_substreams = [(j,k) for j in range(U) for k in range(U)]
        for i in range(U):
            perm_i = all_substreams[:]
            np.random.shuffle(perm_i)
            decode_orders_for_all.append(perm_i)
        candidate_orders.append(decode_orders_for_all)
    
    best_sum_power = np.inf
    best_solution = None
    
    for decode_orders in candidate_orders:
        # Solve for power with this set of decode_orders
        p_candidate = solve_power_allocation_sca(h, sigma, R_min, decode_orders)
        if p_candidate is None:
            # Infeasible or solver didn't converge
            continue
        
        # Check feasibility exactly with the final p_candidate
        feasible, R_matrix = check_feasibility(p_candidate, h, sigma, decode_orders, R_min, U)
        if feasible:
            sum_power = np.sum(p_candidate)
            if sum_power < best_sum_power:
                best_sum_power = sum_power
                best_solution = (decode_orders, p_candidate, R_matrix)
    
    return best_solution

# -----------------------------
# Example usage
if __name__ == "__main__":
    np.random.seed(1)
    
    U = 3  # 3 users, each with 3 sub-streams => total of 9 sub-streams
    # Channel gains h[i,j]: shape (U,U)
    # Let's pick them randomly for demonstration
    h = np.random.uniform(0.5, 2.0, size=(U, U))
    # Noise variances
    sigma = np.ones(U) * 1e-2
    # Min rate requirements per user
    R_min = np.array([3.0, 3.0, 3.0])  # each user needs 3 bits/s/Hz total
    
    solution = solve_minPIC(U, h, sigma, R_min, num_order_candidates=20)
    
    if solution is None:
        print("No feasible solution found under these random orders/approximations.")
    else:
        decode_orders, p_star, R_matrix = solution
        print("=== Best Found Solution ===")
        print("Decoding orders (one per receiver):")
        for i in range(U):
            print(f"Receiver {i} order: {decode_orders[i]}")
        print("\nPower allocation p_star:\n", p_star)
        print("\nSum of powers =", np.sum(p_star))
        # Check final rates:
        for i in range(U):
            user_rate = np.sum(R_matrix[i,:])
            print(f"User {i} total rate = {user_rate:.2f} (min required {R_min[i]})")

ValueError: Problem has an invalid constraint of type <class 'numpy.bool_'>